In [1]:
import geopandas as gpd
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_sample_weight
import os

In [56]:
city = 'AADO'
felyxotp23 = pd.read_pickle('OTPFused/felyxfused' + city)
felyxotp22 = pd.read_pickle('OTPFused/felyxfused' + 'Amsterdam2022')
# PCs = gpd.read_file(os.path.join('..', 'PublicGeoJsons', city + '_PC4.geojson'))
PCs = pd.read_pickle(os.path.join('..', 'PublicGeoJsons', city + '_PC4.geojson'))
pcdata = pd.read_pickle('../PostcodeInfo/PCData')
pcdata.index.names = ['pc4']
pcdata = pcdata.reset_index()

In [57]:
felyxotp22.groupby('month').count()

,carId,fuelLevel,vehicleTypeId,pricingTime,isDamaged,city,time,prev_time,geometry,prev_location,...,pt_modechain,walk_duration,walk_distance,walk_modechain,bike_duration,bike_distance,bike_modechain,car_duration,car_distance,car_modechain
month,,,,,,,,,,,,,,,,,,,,,
7,3727,3727,3727,3727,3727,3727,3727,3727,3727,3727,...,3727,3727,3727,3727,3727,3727,3727,3727,3727,3727
8,3722,3722,3722,3722,3722,3722,3722,3722,3722,3722,...,3722,3722,3722,3722,3722,3722,3722,3722,3722,3722
9,3601,3601,3601,3601,3601,3601,3601,3601,3601,3601,...,3601,3601,3601,3601,3601,3601,3601,3601,3601,3601
10,3722,3722,3722,3722,3722,3722,3722,3722,3722,3722,...,3722,3722,3722,3722,3722,3722,3722,3722,3722,3722
11,3600,3600,3600,3600,3600,3600,3600,3600,3600,3600,...,3600,3600,3600,3600,3600,3600,3600,3600,3600,3600
12,3721,3721,3721,3721,3721,3721,3721,3721,3721,3721,...,3721,3721,3721,3721,3721,3721,3721,3721,3721,3721


In [59]:
felyxotp23.groupby('month').count()

,carId,fuelLevel,vehicleTypeId,pricingTime,isDamaged,city,time,prev_time,geometry,prev_location,...,pt_modechain,walk_duration,walk_distance,walk_modechain,bike_duration,bike_distance,bike_modechain,car_duration,car_distance,car_modechain
month,,,,,,,,,,,,,,,,,,,,,
1,3105,3105,3105,3105,3105,3105,3105,3105,3105,3105,...,3105,3105,3105,3105,3105,3105,3105,3105,3105,3105
2,2800,2800,2800,2800,2800,2800,2800,2800,2800,2800,...,2800,2800,2800,2800,2800,2800,2800,2800,2800,2800
3,3101,3101,3101,3101,3101,3101,3101,3101,3101,3101,...,3101,3101,3101,3101,3101,3101,3101,3101,3101,3101
4,17395,17395,17395,17395,17395,17395,17395,17395,17395,17395,...,17395,17395,17395,17395,17395,17395,17395,17395,17395,17395
5,3102,3102,3102,3102,3102,3102,3102,3102,3102,3102,...,3102,3102,3102,3102,3102,3102,3102,3102,3102,3102
6,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,...,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000


In [60]:
felyxotp = pd.concat([felyxotp23, felyxotp22])
felyxotp.groupby(['month']).count()

,carId,fuelLevel,vehicleTypeId,pricingTime,isDamaged,city,time,prev_time,geometry,prev_location,...,pt_modechain,walk_duration,walk_distance,walk_modechain,bike_duration,bike_distance,bike_modechain,car_duration,car_distance,car_modechain
month,,,,,,,,,,,,,,,,,,,,,
1,3105,3105,3105,3105,3105,3105,3105,3105,3105,3105,...,3105,3105,3105,3105,3105,3105,3105,3105,3105,3105
2,2800,2800,2800,2800,2800,2800,2800,2800,2800,2800,...,2800,2800,2800,2800,2800,2800,2800,2800,2800,2800
3,3101,3101,3101,3101,3101,3101,3101,3101,3101,3101,...,3101,3101,3101,3101,3101,3101,3101,3101,3101,3101
4,17395,17395,17395,17395,17395,17395,17395,17395,17395,17395,...,17395,17395,17395,17395,17395,17395,17395,17395,17395,17395
5,3102,3102,3102,3102,3102,3102,3102,3102,3102,3102,...,3102,3102,3102,3102,3102,3102,3102,3102,3102,3102
6,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,...,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000
7,3727,3727,3727,3727,3727,3727,3727,3727,3727,3727,...,3727,3727,3727,3727,3727,3727,3727,3727,3727,3727
8,3722,3722,3722,3722,3722,3722,3722,3722,3722,3722,...,3722,3722,3722,3722,3722,3722,3722,3722,3722,3722
9,3601,3601,3601,3601,3601,3601,3601,3601,3601,3601,...,3601,3601,3601,3601,3601,3601,3601,3601,3601,3601


In [61]:
felyxotp = felyxotp.replace({-3: np.nan})
felyxotp.head()

,carId,fuelLevel,vehicleTypeId,pricingTime,isDamaged,city,time,prev_time,geometry,prev_location,...,pt_modechain,walk_duration,walk_distance,walk_modechain,bike_duration,bike_distance,bike_modechain,car_duration,car_distance,car_modechain
tripid,,,,,,,,,,,,,,,,,,,,,
100020235312110,1000,90,109,€0.33 / min,False,Amsterdam,2023-05-31 21:10:05+02:00,2023-05-31 20:55:05+02:00,POINT (4.85609 52.37163),POINT (4.85463 52.35017),...,WALK_BUS_WALK,2062.0,2627.116,WALK,622.0,2689.335,BICYCLE,637.0,4442.820,CAR
10002023611341,1000,84,109,€0.33 / min,False,Amsterdam,2023-06-11 03:41:05+02:00,2023-06-11 03:24:05+02:00,POINT (4.89393 52.37891),POINT (4.84561 52.34019),...,WALK_BUS_WALK,4885.0,6210.960,WALK,1467.0,6401.921,BICYCLE,1169.0,9127.600,CAR
10002023621348,1000,56,109,€0.33 / min,False,Amsterdam,2023-06-02 13:48:06+02:00,2023-06-02 13:35:05+02:00,POINT (4.91139 52.36204),POINT (4.89490 52.37080),...,WALK_SUBWAY_WALK,1502.0,1834.903,WALK,486.0,1946.833,BICYCLE,790.0,2648.568,CAR
100420231101053,1004,41,67,€0.30 / min,False,Amsterdam,2023-01-10 10:53:06+01:00,2023-01-10 10:31:06+01:00,POINT (4.87111 52.35553),POINT (4.89552 52.37765),...,WALK_TRAM_WALK,2761.0,3413.321,WALK,864.0,3596.987,BICYCLE,903.0,4018.479,CAR
10042023112127,1004,91,67,€0.30 / min,False,Amsterdam,2023-01-12 12:07:37+01:00,2023-01-12 11:49:38+01:00,POINT (4.89534 52.35146),POINT (4.85382 52.35137),...,WALK_TRAM_WALK_TRAM_WALK,2617.0,3313.670,WALK,768.0,3374.388,BICYCLE,874.0,4231.134,CAR


In [62]:
felyxotp[felyxotp.isna().sum(axis = 1) > 0].distance_moved.describe()

count    25688.000000
mean      2826.692517
std       1807.749864
min        100.090826
25%       1548.258847
50%       2441.579119
75%       3674.822167
max      24756.617217
Name: distance_moved, dtype: float64

In [63]:
felyxotp.distance_moved.describe()

count    54596.000000
mean      2764.710621
std       1740.207299
min        100.090826
25%       1520.285556
50%       2384.865046
75%       3597.417484
max      24756.617217
Name: distance_moved, dtype: float64

In [64]:
felyxotp.isna().sum()

carId                 0
fuelLevel             0
vehicleTypeId         0
pricingTime           0
isDamaged             0
city                  0
time                  0
prev_time             0
geometry              0
prev_location         0
distance_moved        0
timedelta             0
PC4                   0
duration              0
year                  0
month                 0
day                   0
weekday           22093
origin            22093
destination       22093
departuretime     22093
pt_duration        1881
pt_distance        1881
pt_modechain          0
walk_duration       448
walk_distance       448
walk_modechain        0
bike_duration       451
bike_distance       451
bike_modechain        0
car_duration       3907
car_distance       3907
car_modechain         0
dtype: int64

In [65]:
felyxotp.weekday

tripid
100020235312110    4.0
10002023611341     1.0
10002023621348     6.0
100420231101053    3.0
10042023112127     5.0
                  ... 
9982022811649      NaN
9982022831416      NaN
998202288144       NaN
998202288187       NaN
998202294180       NaN
Name: weekday, Length: 54596, dtype: float64

In [66]:
# felyxotp = felyxotp.dropna()

In [67]:
def add_time(df):
    df['seconds'] = df.prev_time.dt.second + 60*(df.prev_time.dt.minute) +3600*(df.prev_time.dt.hour)
    seconds_in_day = 24*60*60
    df['secondsangle'] = (2*np.pi)* (df['seconds']/seconds_in_day)
    df['sin_time'] = np.sin((2*np.pi)*(df.secondsangle))
    df['cos_time'] = np.cos((2*np.pi)*(df.secondsangle))
    df = df.drop(['seconds', 'secondsangle'], axis = 1)
    return df

In [68]:
felyxotp['weekdag'] = felyxotp.prev_time.dt.dayofweek
import holidays
hols = holidays.NL()
felyxotp['feestdag'] = felyxotp['prev_time'].apply(lambda x: x in(hols)).astype(int)
felyxotp.weekdag = felyxotp.weekdag.replace({0:'Maandag', 1:'Dinsdag', 2:'Woensdag', 3:'Donderdag', 4:'Vrijdag', 5:'Zaterdag', 6:'Zondag'})

In [69]:
if city == 'AADO':
    felyxotp = felyxotp.join(
        gpd.GeoDataFrame(
            felyxotp[['prev_location']], geometry=felyxotp[['prev_location']].prev_location,
            crs='EPSG:4326'
        ).drop('prev_location', axis=1).sjoin(PCs.reset_index().rename(columns={'PC4': 'vertpc'})).vertpc
    )
    felyxotp = felyxotp.join(
        gpd.GeoDataFrame(
            felyxotp, geometry=felyxotp.geometry,
            crs='EPSG:4326'
        ).sjoin(PCs.reset_index().rename(columns={'PC4': 'aankpc'})).aankpc
    )

In [70]:
# if city == 'Amsterdam':
#     felyxotp = felyxotp.join(
#         gpd.GeoDataFrame(
#             felyxotp[['prev_location']], geometry=felyxotp[['prev_location']].prev_location,
#             crs='EPSG:4326'
#         ).drop('prev_location', axis=1).sjoin(PCs.rename(columns={'Postcode4': 'vertpc'})).vertpc
#     )
#     felyxotp = felyxotp.join(
#         gpd.GeoDataFrame(
#             felyxotp, geometry=felyxotp.geometry,
#             crs='EPSG:4326'
#         ).sjoin(PCs.rename(columns={'Postcode4': 'aankpc'})).aankpc
#     )
# if city == 'Rotterdam':
#     # PCs = gpd.read_file(os.path.join('..', 'PublicGeoJsons', 'NetherlandsPC4.geojson'))
#     felyxotp = felyxotp.join(
#         gpd.GeoDataFrame(
#             felyxotp[['prev_location']], geometry=felyxotp[['prev_location']].prev_location,
#             crs='EPSG:4326'
#         ).drop('prev_location', axis=1).sjoin(PCs.rename(columns={'pc4_code': 'vertpc'})).vertpc
#     )
#     felyxotp = felyxotp.join(
#         gpd.GeoDataFrame(
#             felyxotp, geometry=felyxotp.geometry,
#             crs='EPSG:4326'
#         ).sjoin(PCs.rename(columns={'pc4_code': 'aankpc'})).aankpc
#     )

In [71]:
# felyxotp['oprijbewijsau'] = [1] * len(felyxotp)
# felyxotp['oprijbewijsmo'] = [1] * len(felyxotp)
felyxotp['CanFelyx'] = [1] * len(felyxotp)

In [72]:
def add_PC4_data(felyxotp):
    felyxotp = felyxotp.merge(pcdata.add_prefix('aank '), left_on = 'aankpc', right_on = 'aank pc4')
    felyxotp = felyxotp.merge(pcdata.add_prefix('vert '), left_on = 'vertpc', right_on = 'vert pc4')
    felyxotp = felyxotp.drop(['aank pc4', 'vert pc4'], axis = 1)
    return felyxotp

In [73]:
felyxotp.duration = round(felyxotp.duration / 60) * 60
felyxotp.duration

tripid
1000202210111720     540.0
1000202210131411     660.0
100020221015036      540.0
1000202210161447    1140.0
1000202210191613     780.0
                     ...  
9982023621729        480.0
9982023626055       1680.0
99820236271213      1080.0
99820236281754      1440.0
9982023651238        720.0
Name: duration, Length: 54960, dtype: float64

In [74]:
# pd.read_pickle('../Odin/OdinWrangled/Odin2018-2021AADO').odin_duration.unique()

In [77]:
felyxotp#.head()

,carId,fuelLevel,vehicleTypeId,pricingTime,isDamaged,city,time,prev_time,geometry,prev_location,...,bike_distance,bike_modechain,car_duration,car_distance,car_modechain,weekdag,feestdag,vertpc,aankpc,CanFelyx
tripid,,,,,,,,,,,,,,,,,,,,,
1000202210111720,1000,99,67,€0.30 / min,False,Amsterdam,2022-10-11 17:20:04+02:00,2022-10-11 17:11:04+02:00,POINT (4.89474 52.34317),POINT (4.87797 52.34705),...,1430.373,BICYCLE,410.0,1983.710,CAR,Dinsdag,0,1077,1078,1
1000202210131411,1000,30,67,€0.30 / min,False,Amsterdam,2022-10-13 14:11:04+02:00,2022-10-13 14:00:04+02:00,POINT (4.88406 52.36556),POINT (4.88126 52.37786),...,1733.690,BICYCLE,582.0,2135.634,CAR,Donderdag,0,1015,1017,1
100020221015036,1000,49,102,€0.30 / min,False,Amsterdam,2022-10-15 00:36:04+02:00,2022-10-15 00:27:04+02:00,POINT (4.88661 52.36151),POINT (4.89745 52.36984),...,1377.731,BICYCLE,NaN,NaN,error: PATH_NOT_FOUND,Zaterdag,0,1012,1017,1
1000202210161447,1000,44,102,€0.30 / min,False,Amsterdam,2022-10-16 14:47:04+02:00,2022-10-16 14:28:04+02:00,POINT (4.84947 52.36956),POINT (4.88822 52.37026),...,3093.306,BICYCLE,954.0,3969.177,CAR,Zondag,0,1012,1057,1
1000202210191613,1000,55,102,€0.30 / min,False,Amsterdam,2022-10-19 16:13:04+02:00,2022-10-19 16:00:04+02:00,POINT (4.88615 52.36495),POINT (4.87439 52.38635),...,3049.808,BICYCLE,894.0,4193.492,CAR,Woensdag,0,1014,1017,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9982023621729,998,49,109,€0.33 / min,False,Amsterdam,2023-06-21 07:29:05+02:00,2023-06-21 07:21:05+02:00,POINT (4.89086 52.34438),POINT (4.89833 52.34527),...,881.192,BICYCLE_WALK,239.0,1163.039,CAR,Woensdag,0,1078,1078,1
9982023626055,998,51,109,€0.33 / min,False,Amsterdam,2023-06-26 00:55:05+02:00,2023-06-26 00:27:05+02:00,POINT (4.89307 52.37656),POINT (4.82190 52.38111),...,5729.407,BICYCLE,1121.0,6583.493,CAR,Maandag,0,1063,1012,1
99820236271213,998,90,109,€0.33 / min,False,Amsterdam,2023-06-27 12:13:05+02:00,2023-06-27 11:55:05+02:00,POINT (4.93275 52.36241),POINT (4.88619 52.38953),...,5188.410,BICYCLE,1231.0,6925.314,CAR,Dinsdag,0,1013,1094,1


In [78]:
def getmeteo():
    meteo19 = pd.read_csv('../Weather/Daily/Amsterdam2019Daily.csv')
    meteo18 = pd.read_csv('../Weather/Daily/Amsterdam2018Daily.csv')
    meteo22 = pd.read_csv('../Weather/Daily/Amsterdam 2022Daily.csv')
    meteo23 = pd.read_csv('../Weather/Daily/Amsterdam2023Daily.csv')
    meteo = pd.concat([meteo18, meteo19, meteo22, meteo23]).drop_duplicates(subset='datetime')
    meteo.datetime = pd.to_datetime(meteo.datetime)
    meteo['date'] = meteo['datetime'].dt.date
    return meteo
def joinmeteo(df, relcols=['windspeed', 'temp', 'feelslike', 'precip', 'precipcover']):
    df['date'] = df['prev_time'].dt.date
    meteo = getmeteo()
    meteojoined = df.reset_index().merge(meteo[['date'] + relcols], on=['date'])
    meteojoined = meteojoined.drop('date', axis = 1)
    meteojoined = meteojoined.set_index('tripid')
    return meteojoined
felyxotp = joinmeteo(felyxotp)
felyxotp.head()

,carId,fuelLevel,vehicleTypeId,pricingTime,isDamaged,city,time,prev_time,geometry,prev_location,...,weekdag,feestdag,vertpc,aankpc,CanFelyx,windspeed,temp,feelslike,precip,precipcover
tripid,,,,,,,,,,,,,,,,,,,,,
1000202210111720,1000,99,67,€0.30 / min,False,Amsterdam,2022-10-11 17:20:04+02:00,2022-10-11 17:11:04+02:00,POINT (4.89474 52.34317),POINT (4.87797 52.34705),...,Dinsdag,0,1077,1078,1,15.7,10.1,10.0,0.0,0.0
102320221011032,1023,47,67,€0.30 / min,False,Amsterdam,2022-10-11 00:32:36+02:00,2022-10-11 00:21:36+02:00,POINT (4.83369 52.36419),POINT (4.80294 52.35990),...,Dinsdag,0,1068,1064,1,15.7,10.1,10.0,0.0,0.0
102320221012012,1023,91,67,€0.30 / min,False,Amsterdam,2022-10-12 00:12:04+02:00,2022-10-11 23:59:04+02:00,POINT (4.89592 52.36706),POINT (4.82686 52.37369),...,Dinsdag,0,1064,1017,1,15.7,10.1,10.0,0.0,0.0
1063202210112130,1063,23,67,€0.30 / min,True,Amsterdam,2022-10-11 21:30:04+02:00,2022-10-11 21:17:04+02:00,POINT (4.93810 52.37391),POINT (4.88626 52.37950),...,Dinsdag,0,1015,1019,1,15.7,10.1,10.0,0.0,0.0
1110202210111930,1110,31,67,€0.30 / min,False,Amsterdam,2022-10-11 19:30:04+02:00,2022-10-11 19:20:04+02:00,POINT (4.86792 52.33483),POINT (4.87126 52.32081),...,Dinsdag,0,1183,1081,1,15.7,10.1,10.0,0.0,0.0


In [79]:
parkingcosts = pd.read_csv('../Misc/pc4_parking_avg_charge.csv')
parkingcosts.PC4 = parkingcosts.PC4.astype(str)
# parkingcosts
felyxotp = felyxotp.drop('PC4', axis = 1).reset_index().merge(parkingcosts, left_on = 'aankpc',  right_on = 'PC4').drop('PC4', axis = 1).set_index('tripid')
felyxotp

,carId,fuelLevel,vehicleTypeId,pricingTime,isDamaged,city,time,prev_time,geometry,prev_location,...,feestdag,vertpc,aankpc,CanFelyx,windspeed,temp,feelslike,precip,precipcover,AVG_CHARGE
tripid,,,,,,,,,,,,,,,,,,,,,
1000202210111720,1000,99,67,€0.30 / min,False,Amsterdam,2022-10-11 17:20:04+02:00,2022-10-11 17:11:04+02:00,POINT (4.89474 52.34317),POINT (4.87797 52.34705),...,0,1077,1078,1,15.7,10.1,10.0,0.000,0.00,0.6791
7022202210111926,7022,70,67,€0.30 / min,False,Amsterdam,2022-10-11 19:26:04+02:00,2022-10-11 19:10:04+02:00,POINT (4.90448 52.34792),POINT (4.86739 52.34832),...,0,1077,1078,1,15.7,10.1,10.0,0.000,0.00,0.6791
8121202210111212,8121,83,67,€0.30 / min,False,Amsterdam,2022-10-11 12:12:37+02:00,2022-10-11 11:52:04+02:00,POINT (4.89494 52.34514),POINT (4.91461 52.35512),...,0,1091,1078,1,15.7,10.1,10.0,0.000,0.00,0.6791
2047202210131915,2047,59,67,€0.30 / min,False,Amsterdam,2022-10-13 19:15:04+02:00,2022-10-13 18:58:04+02:00,POINT (4.89062 52.34081),POINT (4.88579 52.36392),...,0,1017,1078,1,16.3,12.0,12.0,7.061,70.83,0.6791
20832022101358,2083,37,67,€0.30 / min,False,Amsterdam,2022-10-13 05:08:04+02:00,2022-10-13 04:58:04+02:00,POINT (4.89455 52.34356),POINT (4.92033 52.33935),...,0,1096,1078,1,16.3,12.0,12.0,7.061,70.83,0.6791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400520236222152,4005,37,109,€0.33 / min,False,Amsterdam,2023-06-22 21:52:05+02:00,2023-06-22 21:08:05+02:00,POINT (4.78683 52.34957),POINT (4.89153 52.35669),...,0,1072,1060,1,10.8,22.2,21.9,0.000,0.00,0.0000
64092023691846,6409,30,109,€0.33 / min,False,Amsterdam,2023-06-09 18:46:05+02:00,2023-06-09 18:10:05+02:00,POINT (4.78673 52.34953),POINT (4.89292 52.35448),...,0,1072,1060,1,26.5,20.1,20.1,0.000,0.00,0.0000
20582023324125,2058,19,109,€0.33 / min,False,Amsterdam,2023-03-24 01:25:06+01:00,2023-03-24 01:10:06+01:00,POINT (4.78681 52.34961),POINT (4.82517 52.34655),...,0,1066,1060,1,43.0,11.1,10.2,3.634,33.33,0.0000


In [80]:
felyxotp['hvm'] = len(felyxotp) * ['Felyx'] #TODO move to wrangler
felyxotp['khvm'] = len(felyxotp) * ['Felyx']

In [81]:
felyxotp.to_pickle('FelyxModellingData/felyxotp' + city) #16.50 if less than 50

In [82]:
import pandas as pd
import geopandas as gpd
city = 'AADO'
felyxotp = pd.read_pickle('FelyxModellingData/felyxotp' + city)
felyxotp

,carId,fuelLevel,vehicleTypeId,pricingTime,isDamaged,city,time,prev_time,geometry,prev_location,...,aankpc,CanFelyx,windspeed,temp,feelslike,precip,precipcover,AVG_CHARGE,hvm,khvm
tripid,,,,,,,,,,,,,,,,,,,,,
1000202210111720,1000,99,67,€0.30 / min,False,Amsterdam,2022-10-11 17:20:04+02:00,2022-10-11 17:11:04+02:00,POINT (4.89474 52.34317),POINT (4.87797 52.34705),...,1078,1,15.7,10.1,10.0,0.000,0.00,0.6791,Felyx,Felyx
7022202210111926,7022,70,67,€0.30 / min,False,Amsterdam,2022-10-11 19:26:04+02:00,2022-10-11 19:10:04+02:00,POINT (4.90448 52.34792),POINT (4.86739 52.34832),...,1078,1,15.7,10.1,10.0,0.000,0.00,0.6791,Felyx,Felyx
8121202210111212,8121,83,67,€0.30 / min,False,Amsterdam,2022-10-11 12:12:37+02:00,2022-10-11 11:52:04+02:00,POINT (4.89494 52.34514),POINT (4.91461 52.35512),...,1078,1,15.7,10.1,10.0,0.000,0.00,0.6791,Felyx,Felyx
2047202210131915,2047,59,67,€0.30 / min,False,Amsterdam,2022-10-13 19:15:04+02:00,2022-10-13 18:58:04+02:00,POINT (4.89062 52.34081),POINT (4.88579 52.36392),...,1078,1,16.3,12.0,12.0,7.061,70.83,0.6791,Felyx,Felyx
20832022101358,2083,37,67,€0.30 / min,False,Amsterdam,2022-10-13 05:08:04+02:00,2022-10-13 04:58:04+02:00,POINT (4.89455 52.34356),POINT (4.92033 52.33935),...,1078,1,16.3,12.0,12.0,7.061,70.83,0.6791,Felyx,Felyx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400520236222152,4005,37,109,€0.33 / min,False,Amsterdam,2023-06-22 21:52:05+02:00,2023-06-22 21:08:05+02:00,POINT (4.78683 52.34957),POINT (4.89153 52.35669),...,1060,1,10.8,22.2,21.9,0.000,0.00,0.0000,Felyx,Felyx
64092023691846,6409,30,109,€0.33 / min,False,Amsterdam,2023-06-09 18:46:05+02:00,2023-06-09 18:10:05+02:00,POINT (4.78673 52.34953),POINT (4.89292 52.35448),...,1060,1,26.5,20.1,20.1,0.000,0.00,0.0000,Felyx,Felyx
20582023324125,2058,19,109,€0.33 / min,False,Amsterdam,2023-03-24 01:25:06+01:00,2023-03-24 01:10:06+01:00,POINT (4.78681 52.34961),POINT (4.82517 52.34655),...,1060,1,43.0,11.1,10.2,3.634,33.33,0.0000,Felyx,Felyx


In [83]:
def categorize_time(df, datetime_column): #TODO move to wrangler
    # Convert the datetime column to pandas datetime if it's not already
    df[datetime_column] = pd.to_datetime(df[datetime_column])

    # Create a new column to store the time category
    df['time_category'] = 'Other'

     # Define the time ranges
    morning_start = pd.to_datetime('07:00:00').time()
    morning_end = pd.to_datetime('09:00:00').time()
    daytime_start = pd.to_datetime('09:00:00').time()
    daytime_end = pd.to_datetime('16:00:00').time()
    evening_start = pd.to_datetime('16:00:00').time()
    evening_end = pd.to_datetime('18:00:00').time()

    # Categorize the time for each row
    df.loc[(df[datetime_column].dt.time >= morning_start) & (df[datetime_column].dt.time <= morning_end), 'time_category'] = 'Morning'
    df.loc[(df[datetime_column].dt.time > evening_start) & (df[datetime_column].dt.time < evening_end), 'time_category'] = 'Evening'
    df.loc[(df[datetime_column].dt.time >= daytime_start) & (df[datetime_column].dt.time <= daytime_end), 'time_category'] = 'Daytime'

    return df
felyxotp = categorize_time(felyxotp, 'prev_time')

In [84]:
felyxotp.time_category

tripid
1000202210111720    Evening
7022202210111926      Other
8121202210111212    Daytime
2047202210131915      Other
20832022101358        Other
                     ...   
400520236222152       Other
64092023691846        Other
20582023324125        Other
879202321456          Other
3603202210251824    Evening
Name: time_category, Length: 54660, dtype: object

In [85]:
def felyxcost(felyxotp):
    def costmin(x):
        m = min(16.5, x)
        return m
    felyxotp['felyx_cost'] = felyxotp.apply(lambda row: 1 + (row.car_duration * (float(row.pricingTime[1:5])/60)), axis=1)
    felyxotp['felyx_cost'] = felyxotp['felyx_cost'].apply(lambda x: min(25,x))
    felyxotp['felyx_cost'] = felyxotp.apply(lambda row: costmin(row.felyx_cost) if row.car_duration <= 3000 else row.felyx_cost, axis=1)
    # felyxotp.felyx_cost = round(felyxotp.felyx_cost / 10) * 10
    return felyxotp
felyxotp = felyxcost(felyxotp)

In [86]:
felyxotp['felyx_cost']

tripid
1000202210111720     3.0500
7022202210111926     3.8850
8121202210111212     3.9900
2047202210131915     5.8250
20832022101358       3.4600
                     ...   
400520236222152     10.5315
64092023691846       9.7670
20582023324125       5.7520
879202321456         5.9000
3603202210251824     7.6950
Name: felyx_cost, Length: 54660, dtype: float64

In [87]:
def add_costs(felyxotp):
    transit_rate = ((0.147 +0.166)/2) /1000 #perkm
    transit_base = 0.96
    car_rate = 0.34/1000
    felyxotp['pt_cost'] = transit_base + transit_rate * felyxotp['distance_moved']
    felyxotp['car_cost'] = car_rate * felyxotp['car_distance']
    return felyxotp

felyxotp = add_costs(felyxotp)

In [88]:
def GVBprice(df):
    # https://www.amsterdamtips.com/ov-chipkaart
    # APC = [str(x) for x in gpd.read_file('../PublicGeoJsons/AmsterdamPC4.geojson').Postcode4.unique()]
    APC = [str(x) for x in pd.read_pickle('../PublicGeoJsons/AADO_PC4.geojson').index.unique()]
    NetherlandsBoardingFee =  1.08 #35 minutes no boarding fee
    GVB_Distance_Fee = (0.196/1000)#perkm
    max1way = 3.40
    NSAms = 2.60 #3.10 to Bijlmer
    df['pt_cost_GVB'] = NetherlandsBoardingFee + (df['distance_moved'] * GVB_Distance_Fee)
    df['pt_cost_GVB'] = df['pt_cost_GVB'].apply(lambda x: min(3.40,x))
    return df
felyxotp = GVBprice(felyxotp)

In [89]:
def do():
    df.loc[df['aankpc'].isin(APC) & df['vertpc'].isin(APC), 'pt_cost'] = df.loc[df['aankpc'].isin(APC) & df['vertpc'].isin(APC), 'pt_cost_ams']

In [90]:
felyxotp.groupby(['month'])['timedelta'].mean()

month
1    0 days 00:17:00.497952755
2    0 days 00:17:40.901785714
3    0 days 00:16:51.958908507
4    0 days 00:17:24.941253936
5    0 days 00:17:38.214057507
6    0 days 00:18:47.913703703
7    0 days 00:19:19.735163398
8       0 days 00:19:24.767200
9    0 days 00:19:13.619640387
10   0 days 00:19:12.737333333
11   0 days 00:17:45.333055555
12   0 days 00:17:24.772423025
Name: timedelta, dtype: timedelta64[ns]

In [93]:
felyxotp.groupby('month').sample(2500)

,carId,fuelLevel,vehicleTypeId,pricingTime,isDamaged,city,time,prev_time,geometry,prev_location,...,precip,precipcover,AVG_CHARGE,hvm,khvm,time_category,felyx_cost,pt_cost,car_cost,pt_cost_GVB
tripid,,,,,,,,,,,,,,,,,,,,,
119020231191316,1190,45,67,€0.30 / min,False,Amsterdam,2023-01-19 13:16:05+01:00,2023-01-19 12:58:05+01:00,POINT (4.88066 52.37209),POINT (4.88510 52.35633),...,5.815,37.50,2.0930,Felyx,Felyx,Daytime,5.030,1.238517,1.051018,1.428814
118620231191931,1186,56,67,€0.30 / min,False,Amsterdam,2023-01-19 19:31:05+01:00,2023-01-19 19:03:05+01:00,POINT (4.91901 52.34138),POINT (4.86226 52.37118),...,5.815,37.50,2.1278,Felyx,Felyx,Other,7.200,1.756965,4.773124,2.078116
871202312793,871,96,67,€0.30 / min,False,Amsterdam,2023-01-27 09:03:06+01:00,2023-01-27 08:52:06+01:00,POINT (4.87205 52.39669),POINT (4.89739 52.38115),...,0.015,4.17,3.9009,Felyx,Felyx,Morning,3.550,1.342253,0.974450,1.558732
216220231171912,2162,91,67,€0.30 / min,False,Amsterdam,2023-01-17 19:12:05+01:00,2023-01-17 18:45:05+01:00,POINT (4.88003 52.36545),POINT (4.83720 52.38804),...,3.814,29.17,2.0930,Felyx,Felyx,Other,8.195,1.562557,2.496754,1.834641
19842023131858,1984,45,67,€0.30 / min,False,Amsterdam,2023-01-31 08:58:06+01:00,2023-01-31 08:45:06+01:00,POINT (4.86574 52.38613),POINT (4.83733 52.38820),...,0.095,12.50,1.5730,Felyx,Felyx,Morning,4.340,1.264870,1.167867,1.461818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167220221227232,1672,85,67,€0.30 / min,False,Amsterdam,2022-12-27 23:02:05+01:00,2022-12-27 22:47:06+01:00,POINT (4.85087 52.36995),POINT (4.82692 52.35588),...,0.281,16.67,1.3901,Felyx,Felyx,Other,4.115,1.313788,1.482503,1.523083
6554202212171722,6554,44,67,€0.30 / min,False,Amsterdam,2022-12-17 17:22:05+01:00,2022-12-17 17:11:05+01:00,POINT (4.86254 52.37499),POINT (4.85621 52.36206),...,0.000,0.00,0.9932,Felyx,Felyx,Evening,3.530,1.194960,0.849138,1.374263
3695202212102031,3695,62,67,€0.30 / min,False,Amsterdam,2022-12-10 20:31:05+01:00,2022-12-10 20:25:05+01:00,POINT (4.89187 52.34537),POINT (4.87340 52.34170),...,0.000,0.00,0.6791,Felyx,Felyx,Other,3.325,1.167054,0.723843,1.339314


In [3]:
city = 'AADO'
felyxotp = pd.read_pickle('FelyxModellingData/felyxotp' + city)
felyxotp =  felyxotp.groupby('month').sample(2500)
felyxotp.to_pickle('FelyxModellingData/felyxotp' + city)